<div style='float: right'><img src='pic/shakashaka.png'/></div>
## <div id='shakashaka' />シャカシャカ

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
2..2...3..
..........
.........3
.*...0....
..3.......
.....3..3.
2.0.......
.........1
.......1..
2..3...0..""".split('\n')
nw, nh = len(data[0]), len(data)

### 問題
* 盤面のいくつかの白マスを三角形に黒くぬりつぶします
* マスのぬり方は4通りのいずれかです
* 盤面の数字は、その数字の入っているマスにタテヨコに隣り合うマスのうち、三角形にぬるマスの数を表します
* ぬられずに残った部分は、すべて長方形となります

### 変数
* va (1)

### 制約
* $va_{ij}$は１つのみ (2)
* 数字か■なら$va_{ij5}$==1 (3)
* 数字なら周りの斜めの合計と同じ (4)
* 画面端で可能な形の指定 (5)
* 右や下との境で合わない物通しの禁止 (6)
* 白と三角2つの禁止 (7)

In [ ]:
m = LpProblem()
u = np.zeros((nh+2, nw+2, 6), dtype=object)
va = u[1:-1,1:-1] = np.array(addbinvars(nh, nw, 6)) # (1)
w = (u[1:-1,:-2]+u[1:-1,2:]+u[:-2,1:-1]+u[2:,1:-1])[:,:,1:5]
for i, j in product(range(nh), range(nw)):
    v = va[i,j].tolist()
    m += lpSum(v) == 1 # (2)
    if data[i][j] != '.':
        m += v[5] == 1 # (3)
        if data[i][j].isdigit():
            m += lpSum(w[i,j]) == int(data[i][j]) # (4)
    if i == 0:
        m += v[0]+lpSum(v[3:5]) == 0 # (5)
    if j == 0:
        m += v[0]+lpSum(v[2:4]) == 0 # (5)
    if i == nh-1:
        m += v[0]+lpSum(v[1:3]) == 0 # (5)
    if j == nw-1:
        m += v[0]+lpSum(v[1:5:3]) == 0 # (5)
    if i > 0:
        m += lpSum(v[0]+v[3:5]) + lpSum(va[i-1,j,3:6]) <= 1 # 白黒(6)
        m += lpSum(v[1:3]+v[5]) + lpSum(va[i-1,j,:3])  <= 1 # 黒白(6)
    if j > 0:
        m += lpSum(v[0]+v[2:4])   + lpSum(va[i,j-1,2:4])+va[i,j-1,5]   <= 1 # 白黒(6)
        m += lpSum(v[1:5:3]+v[5]) + va[i,j-1,0]+lpSum(va[i,j-1,1:5:3]) <= 1 # 黒白(6)
for i, j in product(range(nh-1), range(nw-1)):
    for ll in [[0,3,0,2],[0,0,3,4],[4,0,1,0],[2,1,0,0]]:
        m += lpSum(va[i+k//2,j+k%2,ll[k]] for k in range(4)) <= 3 # (7)
%time m.solve()
r = np.vectorize(value)(va).astype(int).dot(range(6))
for i in range(nh):
    for j in range(nw):
        print(' %c'%data[i][j] if data[i][j] != '.'
              else '　┛┗┏┓■'[r[i,j]], end='')
    print()